In [2]:
from all_history import *

['C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310', '', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\win32', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\win32\\lib', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\Pythonwin']
G:\My Drive\vepar\output-May-15.xlsx


In [3]:
common_include.unittest = True

In [4]:
h = Historys()

{'common_include.unittest': True}
'Reading History from G:\\My Drive\\vepar\\all_history_test.csv'


In [5]:
@dataclass
class h_test(Historys):
    def __post_init__(self):
        # hist_file = rootdir + "all_history_test.csv"
        super().__post_init__()
        return

ht = h_test()

{'common_include.unittest': True}
'Reading History from G:\\My Drive\\vepar\\all_history_test.csv'


In [62]:
@dataclass
class h_test(Historys):
    def __post_init__(self):
        # hist_file = rootdir + "all_history_test.csv"
        super().__post_init__()
        self.prep_summ_df()
        return
    def prep_summ_df(self):
        # Sample Data (without 'trade_type', using quantity sign to differentiate buy/sell)
        # data = {
        #     'trade_id': [1, 2, 3, 4, 5],
        #     'symbol': ['AAPL', 'GOOG', 'AAPL', 'GOOG', 'AMZN'],
        #     'price': [150.0, 2800.0, 155.0, 2850.0, 3500.0],
        #     'quantity': [10, -5, 15, -8, 2],  # Negative quantity for sell trades
        #     'trade_date': ['2023-05-10', '2023-05-11', '2023-05-12', '2023-05-13', '2023-05-14']
        # }

        # return
        history_trades_df = self.getDF()
        # history_trades_df[Price].replace({'\,': ''})
        history_trades_df['price'] = history_trades_df[Price].replace({'\$': ''}, regex=True).replace({',': ''}, regex=True).astype(float)
        # Convert 'trade_date' to datetime for any date-based operations
        history_trades_df['trade_date'] = pd.to_datetime(history_trades_df[Date])
        trade_count = len(history_trades_df)

        # Group by 'symbol' and aggregate data
        self.summary_df = history_trades_df.groupby(Symbol).agg(
            qty_buy=(Quantity, lambda x: x[x > 0].sum()),  # Sum of positive quantities for buy trades
            qty_sell=(Quantity, lambda x: x[x < 0].sum()),  # Sum of negative quantities for sell trades
            avg_price_buy=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].mean()),
            # Average buy price
            avg_price_sell=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].mean()),
            first_trade_price=('price', 'last'),  # Last trade price
            last_trade_price=('price', 'first'),
            last_trade_qty=(Quantity, 'first'),
            total_buy_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].sum()),
            # # Total buy amount
            total_sell_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].sum()),
            # Average sell price
            # trade_count=('trade_id', 'count'),  # Count of trades
            first_trade_date=('trade_date', 'min'),  # First trade date
            # last_trade_date=('trade_date', 'max')  # Last trade date
            average_sell_price=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].mean())
            , average_buy_price=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].mean())
        ).reset_index()
                # Calculate the gain/loss (considering negative quantity for sell)
        self.summary_df['gain_loss'] = (self.summary_df['avg_price_sell'] - self.summary_df['avg_price_buy']) * abs(
            self.summary_df['qty_sell'])
        return
    
    def _get_float_value(self, sym, field):
        df = ht.summary_df
        x = float(df[df['Symbol'] == sym][field].iloc[0])
        return x
    
    def getFloatValue(self, val):
        return float(val.replace('$', '').replace(',', ''))
    
    def getIntValue(self, val):
        return int(val.replace('$', '').replace(',', ''))
    
    def getSummary(self):
        self.prep_summ_df()
        return self.summary_df

    def getSummaryAttrForSymbolFromDF(self, sym, attr):
        df = self.getSummary()
        if not isinstance(df, DataFrame):
            return None
        df = df.loc[df[Symbol] == sym]
        if df.empty:
            return None
        return df[attr].values[0]
    # def etFloatValueForSymbol(self, sym):
    #     if isinstance(sym, BaseObject):
    #         sym = sym.getBase()
    #     value = self.getSummaryAttrForSymbolFromDF(sym, 'last_trade_price')
    #     return float(value)
    
    def getFloatValueForSymbol(self, sym,  attr):
        if isinstance(sym, BaseObject):
            sym = sym.getBase()
        value = self.getSummaryAttrForSymbolFromDF(sym, attr)
        return float(value,)
    
    def getIntValueForSymbol(self, sym,  attr):
        if isinstance(sym, BaseObject):
            sym = sym.getBase()
        value = self.getSummaryAttrForSymbolFromDF(sym, attr)
        return int(value)

    def getLastTradePriceFor(self, sym):
        return self.getFloatValueForSymbol(sym, 'last_trade_price')
    def getAllSymbols(self):
        return self.all_symbols
    def ifLastTradeBuy(self, sym):
        value = self.lastTradeQty(sym)
        if not value:
            return None
        if value > 0:
            return True
        return False
    def lastTradeQty(self, sym):
        return self.getIntValueForSymbol(sym, 'last_trade_qty')
    def getGainLossForSymbol(self, sym):
        return self.getFloatValueForSymbol(sym, 'gain_loss')
    
ht = h_test()
x = ht.getLastTradePriceFor('AAOI')
x

for sym in ht.getAllSymbols():
    x = ht.getLastTradePriceFor(sym)
    print({sym: x})
    qty, bs, gnl = ht.lastTradeQty(sym), ht.ifLastTradeBuy(sym),  ht.getGainLossForSymbol(sym)
    print({"qty": qty, "bs": bs,  "gnl": gnl})
# ht.summary_df


{'common_include.unittest': True}
'Reading History from G:\\My Drive\\vepar\\all_history_test.csv'
{'CAT': 1345.31}
{'bs': False, 'gnl': nan, 'qty': -1}
{'WDC': 47.2}
{'bs': False, 'gnl': nan, 'qty': -15}
{'BABA': 133.68}
{'bs': False, 'gnl': nan, 'qty': -5}
{'GS': 599.89}
{'bs': False, 'gnl': nan, 'qty': -1}
{'SLB': 36.5}
{'bs': False, 'gnl': nan, 'qty': -12}
{'LEN': 113.0}
{'bs': False, 'gnl': nan, 'qty': -5}
{'IRBT': 2.9}
{'bs': False, 'gnl': nan, 'qty': -110}
{'AMAT': 166.95}
{'bs': False, 'gnl': nan, 'qty': -3}
{'VSCO': 21.5}
{'bs': False, 'gnl': nan, 'qty': -35}
{'PAAS': 23.25}
{'bs': True, 'gnl': 56.699999999999974, 'qty': 20}
{'NVDL': 43.26}
{'bs': False, 'gnl': nan, 'qty': -9}
{'AMZN': 208.96}
{'bs': False, 'gnl': nan, 'qty': -5}
{'AAOI': 16.02}
{'bs': False, 'gnl': 181.1199999999999, 'qty': -35}
{'MU': 92.91}
{'bs': False, 'gnl': nan, 'qty': -6}
{'FAS': 160.35}
{'bs': False, 'gnl': nan, 'qty': -4}
{'CRM': 285.0}
{'bs': False, 'gnl': nan, 'qty': -2}
{'DIS': 110.0}
{'bs': False

In [46]:
x-10

6.02

In [35]:
df = ht.summary_df
x = float(df[df['Symbol'] == 'AAOI']['last_trade_price'].iloc[0])
x 

16.02

In [18]:
h.prep_summ_df()

In [19]:
h.summary_df

,Symbol,qty_buy,qty_sell,avg_price_buy,avg_price_sell,first_trade_price,last_trade_price,last_trade_qty,first_trade_date,last_trade_date,gain_loss
0,AAOI,65,-128,13.775,15.19,13.50,16.02,-35,2025-05-08,2025-05-12,181.12
1,ADBE,0,-1,NaN,395.61,395.61,395.61,-1,2025-05-12,2025-05-12,NaN
2,AFRM,25,-14,48.085,53.40,49.70,46.47,11,2025-05-08,2025-05-09,74.41
3,AI,0,-28,NaN,23.40,23.40,23.40,-28,2025-05-08,2025-05-08,NaN
4,ALIT,0,-55,NaN,5.55,5.55,5.55,-55,2025-05-08,2025-05-08,NaN
...,...,...,...,...,...,...,...,...,...,...,...
72,VECO,0,-24,NaN,21.35,21.35,21.35,-24,2025-05-12,2025-05-12,NaN
73,VKTX,15,0,26.420,NaN,26.42,26.42,15,2025-05-08,2025-05-08,NaN
74,VSCO,0,-35,NaN,21.50,21.50,21.50,-35,2025-05-12,2025-05-12,NaN
75,WDC,0,-30,NaN,47.20,47.20,47.20,-15,2025-05-12,2025-05-12,NaN
